In [ ]:
# pip install boto3
import boto3 # see https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/lightsail.html 

# You input is necessary

* Choose region_name
* paste aws id, aws key and domainName
* Change domain registration according to the name servers listed in the end

In [ ]:
region_name='us-east-1' 
# us-east-2 @ Ohio
# us-east-1 @ N. Virginia
# us-west-2 @ Oregon
# ap-south-1 @ Mumbai
# ap-northeast-2 @ Seoul
# ap-southeast-1 @ Singapore
# ap-southeast-2 @ Sydney
# ap-northeast-1 @ Tokyo
# ca-central-1 @ Central
# eu-central-1 @ Frankfurt
# eu-west-1 @ Ireland
# eu-west-2 @ London
# eu-west-3 @ Paris

In [ ]:
aws_access_key_id='AQQAXE2DCCPDKEBCCLRJ' 
aws_secret_access_key='0006YH0loluHopT/d9c1ccALMsNKZ/tsGrQ3Q9iJN' 

In [ ]:
domainName = 'your.domain.name'

# Just RUN the following and do NOTHING

### Setup Client for the region of N. Virginia

In [ ]:
client = boto3.client('lightsail', region_name=region_name, use_ssl=True, 
                      aws_access_key_id=aws_access_key_id, 
                      aws_secret_access_key=aws_secret_access_key)

### Create Instance in N. Virginia

In [ ]:
response = client.create_instances(
    instanceNames=['created_from_python',],
    availabilityZone=region_name+'a',
    blueprintId='ubuntu_18_04',
    bundleId='micro_2_0',
    userData='echo "success">/home/ubuntu/test.log', # Initialization shellscript
)

### Allocate static ip

In [ ]:
response = client.allocate_static_ip(staticIpName='static_ip_from_python')

### Attach static ip to instance

In [ ]:
response = client.attach_static_ip(
    staticIpName='static_ip_from_python',
    instanceName='created_from_python'
)

##### Get Static ip

In [ ]:
response = client.get_static_ip(
    staticIpName = 'static_ip_from_python'
)
static_ip = response['staticIp']['ipAddress']

### Open port on Firewall 

In [ ]:
response = client.open_instance_public_ports(
    portInfo={
        'fromPort': 443,
        'toPort': 443,
        'protocol': 'tcp',
        'cidrs': [ "0.0.0.0/0", ],
    },
    instanceName='created_from_python'
)

### Create domain resource

In [ ]:
response = client.create_domain(domainName=domainName)

### Create DNS recordset

In [ ]:
response = client.create_domain_entry(
    domainName=domainName,
    domainEntry={ 
        'name': domainName,
        'target': static_ip, 
        'type': 'A', 
    }
)

# Change your domain registration according to these Name Servers.

In [ ]:
[i['target'] for i in client.get_domain( domainName=domainName)['domain']['domainEntries'] if i['type']=='NS']